# Multi-Agent Portfolio Collaboration with OpenAI Agents SDK

## Introduction

*This guide is intended for readers already familiar with OpenAI models and building LLM agents.*

Building a complex, multistep AI system often benefits from **multiple agents working in tandem**. This guide demonstrates how to orchestrate a team of agents using the OpenAI Agents SDK, through the example of a **portfolio research workflow**. Here, specialized agents (Macro, Fundamental, Quantitative) collaborate under a Portfolio Manager agent to solve a challenging investment analysis problem. We focus on high-level architecture, tool usage, and best practices for multi-agent collaboration, with all code and assets drawn from this repo.

By the end of this guide, you'll understand how to design a multi-agent system, choose collaboration patterns, configure various tool types for each agent, and run the system with tracing for observability.

---

## Table of Contents

1. [What is Multi-Agent Collaboration?](#what-is-multi-agent-collaboration)
2. [Collaboration Patterns: Handoff vs. Agent-as-Tool](#collaboration-patterns-handoff-vs-agent-as-tool)
3. [Architecture Overview](#architecture-overview)
4. [Supported Tool Types](#supported-tool-types)
5. [Setup](#setup)
6. [Running the Workflow](#running-the-workflow)
7. [The Head Portfolio Manager (PM) Agent](#the-head-portfolio-manager-pm-agent)
8. [Breaking Down the Head Portfolio Manager Agent](#breaking-down-the-head-portfolio-manager-agent)
9. [Example Output](#example-output)
10. [Best Practices When Building Agents](#best-practices-when-building-agents)
11. [Further Reading & Best Practices](#further-reading--best-practices)

---

## What is Multi-Agent Collaboration?

**Multi-agent collaboration** means multiple autonomous agents (LLM "nodes") coordinate to achieve an overarching goal that would be difficult for a single agent to handle. Instead of one monolithic prompt, each agent handles a specific subtask or expertise area, and an orchestration layer connects these agent "nodes" into a coherent workflow. This approach is useful for complex systems – for example, a financial analysis might be broken into macro-economic analysis, fundamental company analysis, and quantitative signal analysis, each handled by a different agent specialist. The agents share information and their results are combined to produce a final outcome.

### Collaboration Patterns: Handoff vs. Agent-as-Tool

The OpenAI Agents SDK supports multiple patterns for agents to work together:

- **Handoff Collaboration:** One agent can _handoff_ control to another agent mid-problem. In a handoff architecture, each agent knows about the others and can decide when to defer to a more appropriate agent. This is flexible for open-ended or conversational workflows, but can make it harder to maintain a global view of the task. [Read more in the SDK docs.](https://openai.github.io/openai-agents-python/handoffs/)

- **Agent as a Tool:** In this approach, one agent (often a central planner or manager) **calls other agents as if they were tools**. Sub-agents don't take over the conversation; instead, the main agent invokes them for specific subtasks and incorporates their results. This model keeps a single thread of control (the main agent orchestrates everything) and tends to simplify coordination. **This repo uses the agent-as-tool model:** the Portfolio Manager agent remains in charge, using the other specialist agents as tools when it needs their expertise. This choice keeps the overall reasoning transparent and allows parallel execution of sub-tasks, which is ideal for complex analyses.

For more on these collaboration patterns, see the [OpenAI Agents SDK documentation](https://openai.github.io/openai-agents-python/multi_agent/).

---

## Architecture Overview

Our system follows a **hub-and-spoke design**. The **Portfolio Manager agent** is the hub (central coordinator), and the **specialist agents** are the spokes. The user's query (e.g. "How would a planned interest rate reduction affect my GOOGL holdings?") goes first to the Portfolio Manager. The Portfolio Manager agent is prompted to break down the problem and delegate to the appropriate specialist agents. It treats each specialist as a callable tool, invoking them for their portion of the analysis. All three report back to the Portfolio Manager, which then synthesizes a final answer for the user.

![Multi-Agent Investment Report Workflow](static/agent_architecture.png)

---

## Supported Tool Types

A key advantage of the Agents SDK is the flexibility in defining **tools** that agents can use. Tools can range from simple Python functions to external services. In this project, we use:

- **MCP (Model Context Protocol) Server:** Used to connect agents to external tools and data sources in a standardized way. This project uses a local MCP server for Yahoo Finance data (see `mcp/yahoo_finance_server.py`). [Learn more: OpenAI MCP docs](https://openai.github.io/openai-agents-python/mcp/) | [MCP Spec](https://modelcontextprotocol.io/)

- **OpenAI Managed Tools:** Managed tools are built-in, hosted tools provided by OpenAI that require no custom implementation. They offer powerful capabilities out of the box, such as **Code Interpreter** (for quantitative/statistical analysis) and **WebSearch** (for up-to-date news and data). These tools are easy to integrate, maintained by OpenAI, and allow agents to perform advanced actions like code execution and real-time information retrieval without additional setup.

- **Custom Tools:** Custom tools are any Python functions you define and register as tools for your agent. The Agents SDK makes this easy: just decorate your function, and the SDK will automatically extract its name, docstring, and input schema. This is ideal for domain-specific logic, data access, or workflow extensions. 
  In our project, we use custom tools to access FRED economic data ([see FRED API](https://fred.stlouisfed.org/docs/api/api_key.html)) and perform file system operations.

Custom tools give you full flexibility to extend your agent's capabilities beyond built-in or managed tools. [See the SDK docs on function tools.](https://openai.github.io/openai-agents-python/tools/#function-tools)

> **Want to add more tools?** The SDK supports a wide range of tool types, including web search, file search, code execution, and more. [See the full list of supported tools in the SDK documentation.](https://openai.github.io/openai-agents-python/tools/)

---

## Setup

In [ ]:
# Install required dependencies
!pip install -r requirements.txt

**Before running the workflow, set your environment variables:**
- `OPENAI_API_KEY` (for OpenAI access)
- `FRED_API_KEY` (for FRED economic data, see [FRED API key instructions](https://fred.stlouisfed.org/docs/api/api_key.html))

In [ ]:
import os

missing = []
if not os.environ.get('OPENAI_API_KEY'):
    missing.append('OPENAI_API_KEY')
if not os.environ.get('FRED_API_KEY'):
    missing.append('FRED_API_KEY')

if missing:
    print(f"Missing environment variable(s): {', '.join(missing)}. Please set them before running the workflow.")
else:
    print("All required API keys are set.")

---

## Running the Workflow 

Edit the question to whatever you'd like, but keep the date field to improve accuracy!

<div style="border-left: 4px solidrgb(0, 0, 0); padding: 0.5em; background:rgb(255, 229, 229);">
<strong>Disclaimer:</strong> This example is for educational purposes only. Consult a qualified financial professional before making any investment decisions
</div>


The workflow is kicked off by sending a user request to the Head Portfolio Manager (PM) agent. The PM agent orchestrates the entire process, delegating to specialist agents and tools as needed. You can monitor the workflow in real time using OpenAI Traces, which provide detailed visibility into every agent and tool call.

Edit the `question` in the code below to whatever you'd like, but keep the date field to improve accuracy!

<div style="border-left: 4px solid #f39c12; padding: 0.5em; background: #fffbe6;">
<strong>Note:</strong> Depending on the complexity of the task, this request can take up to 10 minutes.
</div>


In [ ]:
import datetime
import json
import os
from pathlib import Path
from contextlib import AsyncExitStack
from agents import Runner, add_trace_processor, trace
from agents.tracing.processors import BatchTraceProcessor
from utils import FileSpanExporter, output_file
from investment_agents.config import build_investment_agents
import asyncio

add_trace_processor(BatchTraceProcessor(FileSpanExporter()))

async def run_workflow():
    if "OPENAI_API_KEY" not in os.environ:
        raise EnvironmentError("OPENAI_API_KEY not set — set it as an environment variable before running.")

    today_str = datetime.date.today().strftime("%B %d, %Y")
    question = (
        f"Today is {today_str}. "
        "How would the planned interest rate reduction effect my holdings in GOOGL if they were to happen? "
        "Considering all the factors effecting its price right now (Macro, Technical, Fundamental, etc.), what is the likelihood of a 7% return by the end of the year? "
        "Give me all the potential scenarios and the likelihood of each one happening. Account for major events like Covid, 2025 Tariffs etc."
    )
    bundle = build_investment_agents()

    async with AsyncExitStack() as stack:
        for agent in [getattr(bundle, "fundamental", None), getattr(bundle, "quant", None)]:
            if agent is None:
                continue
            for server in getattr(agent, "mcp_servers", []):
                await server.connect()
                await stack.enter_async_context(server)

        print("Running multi-agent workflow with tracing enabled...\n")
        with trace(
            "Investment Research Workflow",
            metadata={"question": question[:512]}
        ) as workflow_trace:
            print(
                f"\n🔗 View the trace in the OpenAI console: "
                f"https://platform.openai.com/traces/trace?trace_id={workflow_trace.trace_id}\n"
            )

            response = None
            try:
                response = await asyncio.wait_for(
                    Runner.run(bundle.head_pm, question, max_turns=40),
                    timeout=1200
                )
            except asyncio.TimeoutError:
                print("\n❌ Workflow timed out after 20 minutes.")

            report_path = None
            try:
                if hasattr(response, 'final_output'):
                    output = response.final_output
                    if isinstance(output, str):
                        data = json.loads(output)
                        if isinstance(data, dict) and 'file' in data:
                            report_path = output_file(data['file'])
            except Exception as e:
                print(f"Could not parse investment report path: {e}")

            print(f"Workflow Completed Response from Agent: {response.final_output if hasattr(response, 'final_output') else response}, investment report created: {report_path if report_path else '[unknown]'}")

# In a Jupyter notebook cell, run:
await run_workflow()

**About Traces:**

OpenAI Traces provide a detailed, step-by-step visualization of your agent workflow. When you run the workflow, a trace link is printed in the output (e.g., `https://platform.openai.com/traces/trace?trace_id=...`).

Clicking this link in the OpenAI console lets you:
- See every agent and tool call, including inputs, outputs, and timing.
- Debug and optimize your workflow by inspecting each step.
- Understand how the Head PM agent delegates to specialists and how results are composed.
- Identify bottlenecks, errors, or unexpected behavior in complex multi-agent flows.

Traces are invaluable for observability, debugging, and improving your agent system. Always check the trace after running a workflow to ensure everything is working as expected.

![Example: OpenAI Traces visualization of a multi-agent workflow.](static/trace_example.png)


---

## Breaking Down the Head Portfolio Manager Agent

The Head Portfolio Manager (PM) agent is the orchestrator of the entire workflow. It coordinates a set of four specialist agents, each focused on a different area of expertise. This design is intentional: overloading a single agent with every possible responsibility leads to shallow, generic outputs and makes it hard to maintain or improve your system over time.

### Why This Design?
By breaking the problem into specialized agents—each with a clear role—you get:

- **Deeper, higher-quality research:** Each agent can focus on its domain, using the right tools and prompts for the job. The PM agent brings these perspectives together for a more nuanced, robust answer.

- **Modularity and clarity:** You can update, test, or improve one agent without affecting the others. This makes your system easier to maintain and extend as your needs evolve.

- **Faster results through parallelism:** Independent agents can work at the same time, dramatically reducing the time to complete complex, multi-part analyses.

- **Consistency and auditability:** A structured, prompt-driven workflow ensures every run follows best practices, is easy to debug, and produces outputs you can trust and review.

This approach is ideal for any application where you want depth, specialization, and reliability—whether you're building a research assistant, a decision support tool, or any system that benefits from expert collaboration and orchestration.

**How We Implement This in Practice:**
- Each specialist agent (Fundamental, Macro, Quantitative) is wrapped as a callable tool using the SDK's `function_tool` decorator, with custom names and descriptions. This makes the PM agent's toolset explicit and LLM-friendly.

- The Head PM agent uses the `run_all_specialists_parallel` tool to invoke all three specialists concurrently, leveraging `parallel_tool_calls=True` for maximum speed and efficiency.

- The agent's prompt is loaded from a markdown file (`pm_base.md`), encoding not just the firm's philosophy but also detailed tool usage rules and a step-by-step workflow. This ensures every run is consistent, auditable, and aligned with best practices.

- After gathering and reviewing the specialist outputs, the PM agent uses a dedicated memo editor tool to assemble, format, and finalize the investment report. This separation of concerns keeps the workflow modular and easy to extend.

- The system is designed for extensibility: you can add new specialist agents, swap out tools, or update prompts without breaking the overall orchestration logic. All tool calls, agent decisions, and outputs are captured in OpenAI Traces for full transparency and debugging.

These implementation choices directly support the benefits above—enabling deep, modular, and reliable multi-agent research workflows that are easy to maintain, audit, and improve.

### Head Portfolio Manager Agent: Code

In [ ]:
from agents import Agent, ModelSettings, function_tool
from utils import load_prompt, DISCLAIMER

def build_head_pm_agent(fundamental, macro, quant, memo_edit_tool):
    def make_agent_tool(agent, name, description):
        @function_tool(name_override=name, description_override=description)
        async def agent_tool(input):
            return await specialist_analysis_func(agent, input)
        return agent_tool
    fundamental_tool = make_agent_tool(fundamental, "fundamental_analysis", "Generate the Fundamental Analysis section.")
    macro_tool = make_agent_tool(macro, "macro_analysis", "Generate the Macro Environment section.")
    quant_tool = make_agent_tool(quant, "quantitative_analysis", "Generate the Quantitative Analysis section.")

    @function_tool(name_override="run_all_specialists_parallel", description_override="Run all three specialist analyses (fundamental, macro, quant) in parallel and return their results as a dict.")
    async def run_all_specialists_tool(fundamental_input, macro_input, quant_input):
        return await run_all_specialists_parallel(
            fundamental, macro, quant,
            fundamental_input, macro_input, quant_input
        )

    return Agent(
        name="Head Portfolio Manager Agent",
        instructions=(load_prompt("pm_base.md") + DISCLAIMER),
        model="gpt-4.1",
        tools=[fundamental_tool, macro_tool, quant_tool, memo_edit_tool, run_all_specialists_tool],
        model_settings=ModelSettings(parallel_tool_calls=True, tool_choice="auto", temperature=0)
    )

### The Head PM System Prompt: Enforcing Best Practices

The PM agent's system prompt (see `prompts/pm_base.md`) is the heart of the workflow. It encodes:
- The firm's philosophy (originality, risk awareness, challenging consensus)
- Clear tool usage rules (when to use parallel tools, how to structure inputs)
- A robust, multi-step workflow (determine task type, provide guidance, review outputs, assemble memo, handle missing data)

This prompt ensures that every run is:
- **Consistent:** The same high standards and process are followed every time.
- **Auditable:** Each step, tool call, and decision is visible in the trace.
- **High-Quality:** Outputs are original, risk-aware, and rigorously reviewed.

In [ ]:
# Render the actual system prompt used by the Head Portfolio Manager agent
from pathlib import Path
from IPython.display import Markdown, display

pm_prompt_path = Path("prompts/pm_base.md")
if pm_prompt_path.exists():
    with pm_prompt_path.open("r", encoding="utf-8") as f:
        content = f.read()
    display(Markdown(content))
else:
    print("System prompt not found at prompts/pm_base.md")

## Example Output

Here's an example of an investment report generated through the workflow. See [`static/example_output/investment_report.md`](static/example_output/investment_report.md) for the full output, and the `static/example_output/` folder for referenced images and CSVs.

In [7]:
# Render the actual investment report generated by the workflow
from pathlib import Path
from IPython.display import Markdown, display

report_path = Path("static/example_output/investment_report.md")
if report_path.exists():
    with report_path.open("r", encoding="utf-8") as f:
        content = f.read()
    display(Markdown(content))
else:
    print("No investment report found at static/example_output/investment_report.md")

# Investment Memo: Alphabet (GOOGL) – May 26, 2025

## Executive Summary

Alphabet (GOOGL) presents a compelling, yet nuanced, investment case as of May 2025. The stock trades at $168.47/share, with robust fundamentals, a fortress balance sheet, and strong analyst consensus. However, the probability of achieving a 7% return by year-end is less than 50% in most modeled scenarios, even with a planned interest rate reduction. This thesis is deeply aligned with the firm's vision: it is differentiated, scenario-driven, and risk-aware, challenging the prevailing market optimism by quantifying the material risks from tariffs, regulatory action, and AI competition. While consensus expects a rebound on rate cuts and digital tailwinds, our integrated analysis highlights that macro and idiosyncratic risks could easily derail the bull case. The original insight here is that a rate cut, while directionally positive, is not a panacea for GOOGL, and tail risks remain underappreciated by the market. Both best- and worst-case scenarios are explicitly considered, with scenario probabilities and risk factors supported by historical event studies and current data. This approach embodies the firm's edge: we do not simply follow consensus, but rigorously test it and plan for both upside and downside.

## Fundamentals Perspective

Alphabet's valuation is attractive relative to its history, with a trailing and forward P/E of 18.8, a price-to-book of 5.93, and a price-to-sales of 5.7. Gross margins are 58.6%, and net profit margins are 30.9%. The company's free cash flow for the most recent quarter was $18.95B, and it holds $95.3B in cash and short-term investments against $23.6B in total debt, reflecting a fortress balance sheet. See the latest balance sheet and cash flow data below:

**Balance Sheet (Q1 2025):**

| date       |   Total Debt |   Total Assets |   Total Equity Gross Minority Interest |   Cash Cash Equivalents And Short Term Investments |
|:-----------|-------------:|---------------:|---------------------------------------:|---------------------------------------------------:|
| 2025-03-31 |   2.3564e+10 |    4.75374e+11 |                            3.45267e+11 |                                      9.5328e+10 |

**Cash Flow (Q1 2025):**

| date       |   Free Cash Flow |   Repurchase Of Capital Stock |   End Cash Position |
|:-----------|-----------------:|------------------------------:|--------------------:|
| 2025-03-31 |       1.8953e+10 |                   -1.5068e+10 |          2.3264e+10 |

The business remains resilient, with Q1 2025 revenue of $90.2B and net income of $34.5B. Search and YouTube continue to benefit from digital ad demand, while Cloud is growing at 30% YoY, albeit with some margin pressure. The company's moat is underpinned by dominant platforms, proprietary AI, and massive data scale. However, 2025 tariffs threaten ~7% of search revenue, and regulatory scrutiny is a persistent overhang. Analyst consensus is overwhelmingly bullish, with 15 "Strong Buy," 41 "Buy," and 12 "Hold" ratings, and no "Sell" or "Strong Sell" (see table below):

| period   |   strongBuy |   buy |   hold |   sell |   strongSell |
|:---------|------------:|------:|-------:|-------:|-------------:|
| 0m       |          15 |    41 |     12 |      0 |            0 |

The average target price is $201.47, implying ~20% upside. The original, evidence-based insight here is that while fundamentals are strong and consensus is bullish, the risks from tariffs, regulation, and AI competition are not fully priced in. The scenario analysis below (see Quant section) quantifies these risks. This view is differentiated from consensus by explicitly modeling downside scenarios and not assuming a rate cut alone will deliver the desired return. All data is current as of Q1 2025, with robust coverage of financials, news, and analyst sentiment. The main data gap is the uncertainty around the timing/magnitude of rate cuts and the final form of 2025 tariffs.

## Macro Perspective

The macro environment is characterized by a Federal Funds Rate of 4.33% (April 2025), CPI at 320.3, GDP at $29.98T, and unemployment at 4.2%. The FOMC projects two rate cuts in 2025, totaling 50bps, but the timing is uncertain due to economic and policy risks, especially tariffs. The Trump administration's tariff policies have introduced significant uncertainty, and the Fed has signaled that substantial tariffs could require further rate cuts to counteract economic stagnation. The technology sector, including GOOGL, is sensitive to these macro and policy shifts. 

Tail-risk scenarios are explicitly considered:
- If rate cuts are successful, GOOGL could benefit from a lower cost of capital and sector rotation into tech, supporting the bull case.
- If rate cuts fail to stimulate growth, or if tariffs escalate, GOOGL's ad revenues and international business could be hit, supporting the bear or tail case.
- An unexpected inflation surge could force higher rates, depressing GOOGL's valuation.

The consensus macro view is that rate cuts will support equities, but our variant view is that this is not guaranteed—rate cuts may not be effective if trade tensions and inflation persist. The effectiveness of rate cuts is contingent on broader economic conditions, and the technology sector's sensitivity to global trade dynamics means that tariff escalations could offset the benefits of lower rates. This scenario-driven, risk-aware approach is directly aligned with the firm's vision.

## Quantitative Perspective

Quantitative analysis supports the scenario-driven thesis. Regression analysis (see `googl_interest_rate_event_study_summary.csv`) shows GOOGL's daily returns have a modest negative beta to both 10Y Treasury and Fed Funds Rate changes, consistent with large-cap tech's typical rate sensitivity:

| Variable            | Coefficient | StdErr   | t     | P>\|t\|   |
|:--------------------|:-----------:|:--------:|:-----:|:--------:|
| const               | 0.00794     | 0.00250  | 3.17  | 0.0031   |
| DGS10_Change        | -0.00517    | 0.02822  | -0.18 | 0.856    |
| FEDFUNDS_Change     | -0.03493    | 0.01145  | -3.05 | 0.0043   |
| Event: 2025 Tariffs | 0.01111     | -        | 1     | -        |

Event studies show GOOGL experienced significant negative abnormal returns during Covid and mild underperformance during the 2025 Tariff window. Monte Carlo scenario simulations (see `GOOGL_scenario_sim_summary.csv`) show the probability of achieving a 7% return by year-end 2025 is less than 50% in most scenarios:

| Scenario   |       mean |      std |       min |     max |   prob_>=7% |
|:-----------|-----------:|---------:|----------:|--------:|------------:|
| Bull       |  0.0538    | 0.2645   | -0.638    | 1.550   |      0.4265 |
| Base       |  0.0370    | 0.2578   | -0.643    | 1.462   |      0.4005 |
| Bear       | -0.0110    | 0.2489   | -0.605    | 1.285   |      0.3257 |
| Tail       | -0.1136    | 0.2224   | -0.683    | 1.659   |      0.1891 |

The scenario simulation and event study highlight that even with a rate cut and strong fundamentals, there is a substantial probability (~60%) that GOOGL does **not** achieve a 7% return by year-end, especially if macro or sector-specific risks materialize. This is a more nuanced view than the analyst consensus, and is directly aligned with the firm's vision of differentiated, evidence-based scenario planning. The limitations of the quant analysis are acknowledged: scenario probabilities are model-based, not market-implied, and real-world shocks may be more or less severe than modeled. Key charts and images are embedded below:

![Macro Event Chart](static/example_output/googl_macro_event_chart.png)

![Scenario Simulation Distribution](static/example_output/GOOGL_scenario_sim_distribution.png)

![EPS and Revenue Trends](static/example_output/GOOGL_eps_revenue_trends.png)

## Portfolio Manager Perspective

The PM synthesis is that while a planned interest rate reduction is directionally positive for GOOGL, it is not a panacea. The probability of achieving a 7% return by year-end is less than 50% in most modeled scenarios, with tail risks (tariffs, regulatory shocks, macro surprises) remaining material. The consensus is bullish, but both quant and fundamental analyses highlight that even with strong fundamentals and positive analyst sentiment, macro and idiosyncratic risks could easily derail the bull case. The differentiated, risk-aware thesis here is that GOOGL's exposure to tariffs, regulatory action, and AI competition could cap upside or even lead to underperformance despite rate cuts. Investors should not assume a rate cut alone will deliver the desired return, and should remain vigilant for new macro or sector-specific shocks. The scenario probabilities and risk factors are well-supported by both historical event studies and current data. This is a differentiated, risk-aware thesis that challenges the consensus optimism and is fully aligned with the firm's vision.

## Recommendation & Answer to the Question

The actionable recommendation is to maintain a constructive, but risk-aware, position in GOOGL. While the stock is fundamentally strong and a rate cut is directionally positive, the probability of achieving a 7% return by year-end is less than 50% in most scenarios. This recommendation embodies the firm's vision by being original, evidence-based, and scenario-driven, explicitly planning for both best- and worst-case outcomes. If the recommendation aligns with consensus, it is only because the evidence justifies it; where it diverges, it is because the risks are not fully priced in. The trade-off is between participating in potential upside from rate cuts and digital tailwinds, versus the material risk of underperformance from tariffs, regulation, or macro shocks. Investors should size positions accordingly, monitor macro and policy developments closely, and be prepared to adjust exposure if downside risks materialize.

**END_OF_MEMO**


---

---

## Best Practices When Building Agents

The most effective agentic systems combine modular agent design, clear tool definitions, parallel execution, and structured prompts. This approach—central to the OpenAI Agents SDK—makes your workflows robust, scalable, and easy to debug or extend.

**Key features of the OpenAI Agents SDK that enable these best practices:**
- **Agent loop:** Handles tool calls, LLM reasoning, and workflow control automatically.
- **Python-first orchestration:** Use familiar Python patterns to chain, compose, and orchestrate agents.
- **Handoffs:** Delegate tasks between agents for specialization and modularity.
- **Guardrails:** Validate inputs/outputs and break early on errors for reliability.
- **Function tools:** Register any Python function as a tool, with automatic schema and validation.
- **Tracing:** Visualize, debug, and monitor every step of your workflow for full transparency.

A combination of well-designed tools, thoughtful orchestration, and careful model selection is crucial for building effective agent systems. In this example, we use the GPT-4.1 family of models for their strong analytical and tool-use capabilities ([see the GPT-4.1 Prompting Guide](https://cookbook.openai.com/examples/gpt4-1_prompting_guide)). For deeper architectural best practices, see the included [A Practical Guide to Building Agents (PDF)](https://cdn.openai.com/business-guides-and-resources/a-practical-guide-to-building-agents.pdf). By bringing these elements together, you get a system that is robust, scalable, and easy to debug or extend.

Please try out the sample with your own investment questions, and please share any feedback! Happy building.

---

## Further Reading & Best Practices

- [OpenAI Agents SDK Documentation](https://openai.github.io/openai-agents-python/)
- [OpenAI Agents SDK: Multi-Agent Orchestration](https://openai.github.io/openai-agents-python/multi_agent/)
- [OpenAI Agents SDK: Tool List](https://openai.github.io/openai-agents-python/tools/)
- [OpenAI Agents SDK: MCP Documentation](https://openai.github.io/openai-agents-python/mcp/)

- [MCP Spec](https://spec.modelcontextprotocol.io/specification/2024-11-05/architecture/)
- [OpenAI Cookbook](https://github.com/openai/openai-cookbook)
- ([GPT-4.1 Prompting Guide](https://cookbook.openai.com/examples/gpt4-1_prompting_guide))
- [A Practical Guide to Building Agents (PDF)](https://cdn.openai.com/business-guides-and-resources/a-practical-guide-to-building-agents.pdf)

---

``` 